#**antsys**
A general purpose ant colony optimization system.
<br/><br/>**Overview**
<br/>The Ant Colony Optimization (ACO) is a technique, inspired by the foraging behavior of ants, to find good solutions for discrete optimization problems. Its central metaphor resides in the indirect communication mechanism through chemical signals (pheromones) used by many species of social ants in their search for food sources.
<br/>The same inspiration was build in the **antsys** package, wich takes advantage of *python* flexibility to be easily applied to different optimization problems.
<br/><br/>**Installation**
<br/>Installation via ```pip```

In [1]:
!pip3 install antsys

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Usage Example:** *Knapsack Problem*
<br/>The **antsys** package was designed to be easy to apply to different optimization problems. So, the *knapsack problem* was selected as example, since the ACO's application is not straightforward. This problem is based on a situation in which, from a set of objects of different weights and values, you want to fill a *knapsack* obtaining the highest possible value without exceeding its capacity.
<br/>1 - Import necessary packages and modules

In [2]:
from antsys import AntWorld
from antsys import AntSystem
import numpy as np
import random

2 - Generate a knapsack problem instance

In [9]:
# generate and show knapsack capacity
max_weight = random.randint(200,500)
print('knapsack max weight =', max_weight)

# generate and show available items
print('\navailable items:')
print('|item|weight| value|')
items = []
for i in range(20):
  items.append((i, random.randint(50,200), random.randint(100,500)))
  print('|%4i|%6i|%6i|' % items[i])

knapsack max weight = 376

available items:
|item|weight| value|
|   0|   169|   375|
|   1|   143|   158|
|   2|   118|   306|
|   3|   174|   383|
|   4|   143|   223|
|   5|   124|   267|
|   6|   190|   110|
|   7|   199|   360|
|   8|   157|   200|
|   9|   131|   311|
|  10|   109|   448|
|  11|   150|   438|
|  12|   158|   141|
|  13|    56|   130|
|  14|   156|   427|
|  15|    68|   393|
|  16|   174|   271|
|  17|    65|   498|
|  18|    79|   299|
|  19|   184|   417|


3 - The function ```knapsack_rules``` will append information to the edges during the world creation. In this case there will be two edges between pairs of nodes, one assigning (```1```) and the other discharging (```0```) the item (```end```) of being included in the knapsack.

In [32]:
def knapsack_rules(start, end):
  return [0, 1]

4 - The function ```knapsack_cost``` will be used to calculate the cost of any possible solution (```path```).

In [33]:
def knapsack_cost(path):
  k_value = 0
  k_weight = 0
  for edge in path:
    if edge.info == 1:
      k_value += edge.end[2]
      k_weight += edge.end[1]
  cost = 1/k_value
  if k_weight > max_weight:
    cost += 1
  else:
    for edge in path:
      if edge.info == 0 and edge.end[1] <= (max_weight-k_weight):
        cost += 1
  return cost

5 - The ```knapsack_heuristic``` is a simple heuristic that will help the ants to make better choices. The probability to choose an item that fits in the remaining capacity of the knapsack will be slightly higher.


In [20]:
def knapsack_heuristic(path, candidate):
  k_weight = 0
  for edge in path:
    if edge.info == 1:
      k_weight += edge.end[1]
  if candidate.info == 1 and candidate.end[1] < (max_weight-k_weight):
    return 0
  elif candidate.info == 0:
    return 1
  else:
    return 2

6 - This function shows the details of a possible solution (```path```).

In [21]:
def print_solution(path):
  print('knapsack items:')
  print('|item|weight| value|')
  value = 0
  weight = 0
  for edge in path:
    if(edge.info == 1):
      print('|%4i|%6i|%6i|' % edge.end)
      value+=edge.end[2]
      weight+=edge.end[1]
  print('total weight = %g\ntotal value = %g' % (weight, value))

7 - The world (```new_world```) is created from the nodes (```items```) as a non-complete graph. In this point, ```knapsack_rules```, ```knapsack_cost``` and ```knapscack_heuristic``` are defined as respectively ```r_func```, ```c_func``` and ```h_func```. These functions are bound to the world and the first one has an important role in its structure.

In [29]:
new_world = AntWorld(items, knapsack_rules, knapsack_cost, knapsack_heuristic, False, 100)

8 - Configure ```ant_opt``` as an ```AntSystem```.

In [34]:
ant_opt = AntSystem(world=new_world, n_ants=100)

9 - Execute the optimization loop.

In [35]:
ant_opt.optimize(100,50)

| iter |         min        |         max        |        best        |
|     1|         0.000690608|             1.00164|         0.000690608|
|     2|         0.000618429|             1.00265|         0.000618429|
|     3|         0.000690608|             2.00117|         0.000618429|
|     4|         0.000606061|             4.00122|         0.000606061|
|     5|         0.000618429|             1.00265|         0.000606061|
|     6|         0.000618429|             1.00138|         0.000606061|
|     7|         0.000618429|             2.00151|         0.000606061|
|     8|         0.000618429|             1.00265|         0.000606061|
|     9|         0.000618429|             2.00117|         0.000606061|
|    10|         0.000606061|             2.00082|         0.000606061|
|    11|         0.000618429|             1.00227|         0.000606061|
|    12|         0.000618429|             1.00213|         0.000606061|
|    13|         0.000618429|             3.00116|         0.000

10 - Show details about the best solution found.

In [36]:
print('\nknapsack max weight =', max_weight)
print_solution(ant_opt.g_best[2])


knapsack max weight = 376
knapsack items:
|item|weight| value|
|   9|   131|   311|
|  10|   109|   448|
|  15|    68|   393|
|  17|    65|   498|
total weight = 373
total value = 1650
